### Problem 2: Momentum factor
Code and notes for subproblem 2: Farma-French Approach

In [636]:
# packages and load data.
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import Utils
from scipy.optimize import minimize 
from sklearn.linear_model import LinearRegression as lm
import statsmodels.api as sm
import Backtest as bt   
import matplotlib.dates as mdates

FF = pd.read_csv("Data_clean/FF_cleaned.csv")
bond = pd.read_csv("Data_clean/bond_returns.csv")
FF["RF"] = FF["RF"] /100 
MOMexp = pd.read_csv("Data_clean/6_Portfolios_ME_Prior_12_2_returns.csv")
MOMdep = pd.read_csv("Data_clean/25_Portfolios_ME_Prior_12_2_returns.csv")
MOMexp["Market Return"] = MOMexp["Market Return"] / 100
MOMdep["Market Return"] = MOMdep["Market Return"] / 100

# Keep only RF to create new SMB and MOM
MOMexp = pd.merge(MOMexp, FF[["Date","RF"]], 'left', on = "Date")
MOMdep = pd.merge(MOMdep, FF[["Date","RF"]], 'left', on = "Date")
# MOMdep = pd.merge(MOMdep, bond[["Date","10YrReturns"]], 'left', on = 'Date' )

### Format Data
That is, we construct SMB and MOM in MOMexp

In [637]:
MOMexp["SMB"] = (MOMexp["SMALL LoPRIOR"]  + MOMexp["ME1 PRIOR2"] + 
                 MOMexp["SMALL HiPRIOR"] ) / 3 + (MOMexp["BIG LoPRIOR"]  - 
                            MOMexp["ME2 PRIOR2"] + MOMexp["BIG HiPRIOR"] ) / 3 

MOMexp["MOM"] = (MOMexp["SMALL HiPRIOR"] + 
                 MOMexp["BIG HiPRIOR"]) / 2 - (MOMexp["SMALL LoPRIOR"] + MOMexp["BIG LoPRIOR"]) / 2 
# SMB and MOM is returns - normalize to fit returns on market
MOMexp["MOM"],MOMexp["SMB"] =MOMexp["MOM"] / 100,MOMexp["SMB"] / 100

MOMexp["Mkt-RF"] = MOMexp["Market Return"] - MOMexp["RF"]
# Clean data:
MOMexp = MOMexp[["Date","Mkt-RF", "SMB", "MOM"]]

### Regressions
We are now ready to do the regressions.

In [638]:
# One could for starters mimic FF original analysis using RF DATA


In [639]:
# Regression: 
def regression(dep,exp,rf):
    var = dep.columns.drop(["Date", "Market Return", "RF"])
    params = pd.DataFrame(columns =["a", "b","s","m","R2/s(e)"])
    tvals =  pd.DataFrame(columns =["a", "b","s","m", "R2/s(e)"])
    X = np.array(exp).reshape(len(exp),len(exp.columns))
    X = sm.add_constant(X)
    for v in var:
        y = np.array(dep[v]) / 100 - np.array(rf["RF"])
        linmod = sm.OLS(y,X)
        results = linmod.fit()
        coef = pd.DataFrame([np.append(results.params.round(2),results.rsquared.round(2))],
                            columns=["a", "b","s","m","R2/s(e)"])
        ttest = pd.DataFrame([np.append(results.tvalues.round(2),np.std(results.resid).round(4))],
                            columns=["a", "b","s","m", "R2/s(e)"])
        params = pd.concat([params,coef], ignore_index=True)
        tvals =  pd.concat([tvals,ttest], ignore_index=True)
    return params, tvals

# params # Relatively close - > So assume it is correct.

In [640]:
# Create output table: 
# Generate table:
def out_array(param,dep,exp,rf):
    p = param
    params, tvals = regression(dep,exp,rf)
    N = np.sqrt(len(params[p]))
    parameter = np.array(params[p]).reshape((int(N),int(N)))
    ttest = np.array(tvals[p]).reshape((int(N),int(N)))
    tab = np.append(parameter,ttest,axis=1)
    tabdf = pd.DataFrame(tab, columns=pd.MultiIndex.from_tuples(
                          [(r"\beta/R^2","LPrior"), ("","2"), ("","3"), ("","4"), ("","HPrior"),
                           (r"t(\beta)/s(e)","LPrior"), ("","2"), ("","3"), ("","4"), ("","HPrior")]))
    tabdf = tabdf.set_index([pd.Index([param,"", "", "", ""]),pd.Index(["S", "2", "3", "4", "B"])])

    return tabdf

dep = MOMdep.copy()
exp = MOMexp.copy()
rf = FF.copy()
exp = exp.drop(columns = "Date")
rf = FF.drop(columns = "Date")

a = out_array("a", dep, exp, rf)
b = out_array("b", dep, exp, rf)
s = out_array("s", dep, exp, rf)
m = out_array("m", dep, exp, rf)
R2_se = out_array("R2/s(e)", dep, exp, rf)
tab = pd.concat([a,b,s,m, R2_se])


C:\Users\andre\AppData\Local\Temp\ipykernel_19148\1946023651.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  params = pd.concat([params,coef], ignore_index=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_19148\1946023651.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tvals =  pd.concat([tvals,ttest], ignore_index=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_19148\1946023651.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated.

In [641]:
# To latex regression tab:
latex_table = tab.to_latex(index=True, float_format="%.2f", 
                           caption=r"Regression: R(t)-RF(t)=a + b [RM(t) - RF(t)] + sSMB(t) + m MOM(t) +e(t)", 
                           label="tab:regression")
print(latex_table)


\begin{table}
\caption{Regression: R(t)-RF(t)=a + b [RM(t) - RF(t)] + sSMB(t) + m MOM(t) +e(t)}
\label{tab:regression}
\begin{tabular}{llrrrrrrrrrr}
\toprule
 &  & \beta/R^2 & \multicolumn{4}{r}{} & t(\beta)/s(e) & \multicolumn{4}{r}{} \\
 &  & LPrior & 2 & 3 & 4 & HPrior & LPrior & 2 & 3 & 4 & HPrior \\
\midrule
a & S & -0.01 & -0.00 & -0.00 & -0.00 & -0.00 & -6.45 & -2.21 & -0.80 & -2.10 & -2.74 \\
\cline{1-12}
\multirow[t]{4}{*}{} & 2 & -0.01 & -0.00 & -0.00 & -0.00 & -0.00 & -10.10 & -3.66 & -2.37 & -2.81 & -5.73 \\
 & 3 & -0.00 & -0.00 & -0.00 & -0.00 & -0.00 & -6.35 & -1.60 & -1.51 & -3.46 & -4.42 \\
 & 4 & -0.00 & 0.00 & 0.00 & -0.00 & -0.00 & -2.41 & 0.36 & 0.83 & -0.57 & -2.19 \\
 & B & 0.00 & 0.00 & 0.00 & 0.00 & -0.00 & 0.53 & 4.12 & 3.26 & 2.05 & -2.45 \\
\cline{1-12}
b & S & -0.94 & -0.90 & -0.84 & -0.75 & -0.75 & -18.29 & -17.67 & -17.47 & -13.59 & -14.47 \\
\cline{1-12}
\multirow[t]{4}{*}{} & 2 & -0.24 & -0.29 & -0.16 & -0.25 & -0.33 & -7.63 & -9.57 & -5.01 & -7.68 & -11

Now for the Subperiods. 
192701-196306:

In [642]:
dep = MOMdep.copy()
exp = MOMexp.copy()
rf = FF.copy()
dep = dep[(dep["Date"]<'1963-07-31')]
exp = exp[(exp["Date"]<'1963-07-31')].drop(columns = "Date")
rf = rf[( rf["Date"]<'1963-07-31')].drop(columns = "Date")


a = out_array("a", dep, exp, rf)
b = out_array("b", dep, exp, rf)
s = out_array("s", dep, exp, rf)
m = out_array("m", dep, exp, rf)
R2_se = out_array("R2/s(e)", dep, exp, rf)
tab = pd.concat([a,b,s,m, R2_se])

# To latex regression tab:
latex_table = tab.to_latex(index=True, float_format="%.2f", 
                           caption=r"Subperiod: 192701-196306, Regression: R(t)-RF(t)=a + b [RM(t) - RF(t)] + sSMB(t) + m MOM(t) +e(t)", 
                           label="tab:regression")
print(latex_table)

C:\Users\andre\AppData\Local\Temp\ipykernel_19148\1946023651.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  params = pd.concat([params,coef], ignore_index=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_19148\1946023651.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tvals =  pd.concat([tvals,ttest], ignore_index=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_19148\1946023651.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated.

\begin{table}
\caption{Subperiod: 192701-196306, Regression: R(t)-RF(t)=a + b [RM(t) - RF(t)] + sSMB(t) + m MOM(t) +e(t)}
\label{tab:regression}
\begin{tabular}{llrrrrrrrrrr}
\toprule
 &  & \beta/R^2 & \multicolumn{4}{r}{} & t(\beta)/s(e) & \multicolumn{4}{r}{} \\
 &  & LPrior & 2 & 3 & 4 & HPrior & LPrior & 2 & 3 & 4 & HPrior \\
\midrule
a & S & 0.00 & 0.00 & 0.00 & -0.00 & 0.00 & 1.12 & 1.75 & 2.47 & -0.05 & 0.11 \\
\cline{1-12}
\multirow[t]{4}{*}{} & 2 & -0.00 & -0.00 & -0.00 & -0.00 & -0.00 & -2.56 & -0.41 & -0.29 & -0.85 & -0.52 \\
 & 3 & -0.00 & 0.00 & -0.00 & -0.00 & -0.00 & -3.34 & 0.12 & -0.58 & -1.25 & -0.97 \\
 & 4 & -0.00 & -0.00 & 0.00 & -0.00 & 0.00 & -0.48 & -1.21 & 0.22 & -0.78 & 1.27 \\
 & B & 0.00 & -0.00 & 0.00 & 0.00 & -0.00 & 0.32 & -0.13 & 2.74 & 1.94 & -0.64 \\
\cline{1-12}
b & S & -1.31 & -1.40 & -1.38 & -1.13 & -0.95 & -13.85 & -15.13 & -16.62 & -10.56 & -8.68 \\
\cline{1-12}
\multirow[t]{4}{*}{} & 2 & -0.36 & -0.50 & -0.29 & -0.41 & -0.46 & -6.71 & -10.24 & -5

C:\Users\andre\AppData\Local\Temp\ipykernel_19148\1946023651.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  params = pd.concat([params,coef], ignore_index=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_19148\1946023651.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tvals =  pd.concat([tvals,ttest], ignore_index=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_19148\1946023651.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated.

196307-199112


In [643]:
dep = MOMdep.copy()
exp = MOMexp.copy()
rf = FF.copy()
dep = dep[(dep["Date"]>='1963-07-31') & (dep["Date"]<='1991-12-31')]
exp = exp[(exp["Date"]>='1963-07-31')& (exp["Date"]<='1991-12-31')].drop(columns = "Date")
rf = rf[( rf["Date"]>='1963-07-31')& (rf["Date"]<='1991-12-31')].drop(columns = "Date")


a = out_array("a", dep, exp, rf)
b = out_array("b", dep, exp, rf)
s = out_array("s", dep, exp, rf)
m = out_array("m", dep, exp, rf)
R2_se = out_array("R2/s(e)", dep, exp, rf)
tab = pd.concat([a,b,s,m, R2_se])

# To latex regression tab:
latex_table = tab.to_latex(index=True, float_format="%.2f", 
                           caption=r"Subperiod: 196307-199112, Regression: R(t)-RF(t)=a + b [RM(t) - RF(t)] + sSMB(t) + m MOM(t) +e(t)", 
                           label="tab:regression")
print(latex_table)

C:\Users\andre\AppData\Local\Temp\ipykernel_19148\1946023651.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  params = pd.concat([params,coef], ignore_index=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_19148\1946023651.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tvals =  pd.concat([tvals,ttest], ignore_index=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_19148\1946023651.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated.

\begin{table}
\caption{Subperiod: 196307-199112, Regression: R(t)-RF(t)=a + b [RM(t) - RF(t)] + sSMB(t) + m MOM(t) +e(t)}
\label{tab:regression}
\begin{tabular}{llrrrrrrrrrr}
\toprule
 &  & \beta/R^2 & \multicolumn{4}{r}{} & t(\beta)/s(e) & \multicolumn{4}{r}{} \\
 &  & LPrior & 2 & 3 & 4 & HPrior & LPrior & 2 & 3 & 4 & HPrior \\
\midrule
a & S & -0.02 & -0.01 & -0.01 & -0.01 & -0.01 & -14.37 & -7.58 & -6.06 & -7.57 & -8.10 \\
\cline{1-12}
\multirow[t]{4}{*}{} & 2 & -0.01 & -0.01 & -0.00 & -0.00 & -0.01 & -14.13 & -6.23 & -4.76 & -5.11 & -7.93 \\
 & 3 & -0.01 & -0.00 & -0.00 & -0.00 & -0.00 & -6.77 & -2.97 & -3.13 & -5.35 & -4.27 \\
 & 4 & -0.00 & 0.00 & -0.00 & -0.00 & -0.00 & -2.66 & 0.78 & -1.32 & -1.11 & -3.45 \\
 & B & 0.00 & 0.01 & 0.00 & -0.00 & -0.00 & 1.36 & 5.73 & 2.12 & -0.08 & -2.83 \\
\cline{1-12}
b & S & -0.77 & -0.59 & -0.56 & -0.49 & -0.62 & -11.91 & -11.27 & -10.61 & -9.76 & -11.50 \\
\cline{1-12}
\multirow[t]{4}{*}{} & 2 & -0.14 & -0.07 & -0.01 & -0.08 & -0.14 & -2.78

C:\Users\andre\AppData\Local\Temp\ipykernel_19148\1946023651.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  params = pd.concat([params,coef], ignore_index=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_19148\1946023651.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tvals =  pd.concat([tvals,ttest], ignore_index=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_19148\1946023651.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated.

199201-202312

In [644]:
dep = MOMdep.copy()
exp = MOMexp.copy()
rf = FF.copy()
dep = dep[(dep["Date"]>='1992-01-31') & (dep["Date"]<='2023-12-31')]
exp = exp[(exp["Date"]>='1992-01-31')& (exp["Date"]<='2023-12-31')].drop(columns = "Date")
rf = rf[( rf["Date"]>='1992-01-31')& (rf["Date"]<='2023-12-31')].drop(columns = "Date")


a = out_array("a", dep, exp, rf)
b = out_array("b", dep, exp, rf)
s = out_array("s", dep, exp, rf)
m = out_array("m", dep, exp, rf)
R2_se = out_array("R2/s(e)", dep, exp, rf)
tab = pd.concat([a,b,s,m, R2_se])

# To latex regression tab:
latex_table = tab.to_latex(index=True, float_format="%.2f", 
                           caption=r"Subperiod: 199201-202312, Regression: R(t)-RF(t)=a + b [RM(t) - RF(t)] + sSMB(t) + m MOM(t) +e(t)", 
                           label="tab:regression")
print(latex_table)

C:\Users\andre\AppData\Local\Temp\ipykernel_19148\1946023651.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  params = pd.concat([params,coef], ignore_index=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_19148\1946023651.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tvals =  pd.concat([tvals,ttest], ignore_index=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_19148\1946023651.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated.

\begin{table}
\caption{Subperiod: 199201-202312, Regression: R(t)-RF(t)=a + b [RM(t) - RF(t)] + sSMB(t) + m MOM(t) +e(t)}
\label{tab:regression}
\begin{tabular}{llrrrrrrrrrr}
\toprule
 &  & \beta/R^2 & \multicolumn{4}{r}{} & t(\beta)/s(e) & \multicolumn{4}{r}{} \\
 &  & LPrior & 2 & 3 & 4 & HPrior & LPrior & 2 & 3 & 4 & HPrior \\
\midrule
a & S & -0.01 & -0.00 & 0.00 & 0.00 & -0.00 & -4.68 & -1.29 & 0.64 & 1.79 & -0.69 \\
\cline{1-12}
\multirow[t]{4}{*}{} & 2 & -0.00 & 0.00 & 0.00 & 0.00 & -0.00 & -4.57 & 0.13 & 0.67 & 0.61 & -3.04 \\
 & 3 & -0.00 & -0.00 & 0.00 & -0.00 & -0.00 & -2.08 & -0.04 & 0.97 & -0.52 & -4.15 \\
 & 4 & -0.00 & 0.00 & 0.00 & 0.00 & -0.00 & -2.45 & 1.90 & 2.42 & 0.95 & -2.90 \\
 & B & -0.00 & 0.00 & 0.00 & 0.00 & -0.00 & -0.47 & 3.29 & 2.31 & 1.61 & -1.74 \\
\cline{1-12}
b & S & -0.66 & -0.42 & -0.29 & -0.42 & -0.61 & -8.21 & -8.14 & -5.54 & -7.42 & -10.03 \\
\cline{1-12}
\multirow[t]{4}{*}{} & 2 & -0.23 & -0.16 & -0.10 & -0.19 & -0.33 & -4.43 & -3.23 & -2.03 & -4

C:\Users\andre\AppData\Local\Temp\ipykernel_19148\1946023651.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  params = pd.concat([params,coef], ignore_index=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_19148\1946023651.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tvals =  pd.concat([tvals,ttest], ignore_index=True)


### Appendix: Replicate tables and similarly from FF.
Here, we produce code to do things similarly to the original paper.

In [645]:
# Take july 1963 to december 1991
OFF = FF[(FF["Date"]>='1963-07-31') & ( FF["Date"]<'1992-01-31')] 
OFF["Mkt-RF"], OFF["SMB"], OFF["HML"] = OFF["Mkt-RF"]/100, OFF["SMB"]/100, OFF["HML"] /100
RF = OFF["RF"]
OFF = OFF[["Mkt-RF", "SMB", "HML"]]

C:\Users\andre\AppData\Local\Temp\ipykernel_19148\3164172233.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  OFF["Mkt-RF"], OFF["SMB"], OFF["HML"] = OFF["Mkt-RF"]/100, OFF["SMB"]/100, OFF["HML"] /100


In [646]:
# Read subparts of FF data. 
pf_25_vwr = pd.read_csv("Data/25_Portfolios_5x5.csv", skiprows=15, nrows = 1171)
pf_25_vwr = pf_25_vwr.rename(columns = {"Unnamed: 0" : "Date"})
pf_25_vwr["Date"] = pd.to_datetime(pf_25_vwr["Date"], format="%Y%m") + pd.offsets.MonthEnd()
pf_25_vwr = pf_25_vwr[(pf_25_vwr["Date"]<pd.Timestamp('1992-01-31')) & (pf_25_vwr["Date"]>=pd.Timestamp('1963-07-31'))] 
pf_25_vwr = pf_25_vwr.set_index("Date").sort_index(ascending=True).reset_index()

pf_25_nf = pd.read_csv("Data/25_Portfolios_5x5.csv", skiprows=2581, nrows = 1171)
pf_25_nf = pf_25_nf.rename(columns = {"Unnamed: 0" : "Date"})
pf_25_nf["Date"] = pd.to_datetime(pf_25_nf["Date"], format="%Y%m") + pd.offsets.MonthEnd()
pf_25_nf = pf_25_nf[(pf_25_nf["Date"]<pd.Timestamp('1992-01-31')) & (pf_25_nf["Date"]>=pd.Timestamp('1963-07-31'))] 
pf_25_nf = pf_25_nf.set_index("Date").sort_index(ascending=True).reset_index()

pf_25_nf["Y"] =  pf_25_nf['Date'].dt.year


In [647]:
# Subtable 1
pf_25_nf = pf_25_nf.groupby(["Y"]).mean()
pf_25_nf = pf_25_nf.rename({"ME1 BM2": "SMALL BM2", "ME1 BM3": "SMALL BM3", "ME1 BM4": "SMALL BM4",
                            "ME5 BM2": "BIG BM2", "ME5 BM3": "BIG BM3", "ME5 BM4": "BIG BM4"})


row = ["SMALL", "ME1", "ME2", "ME3", "BIG"]


In [648]:
# Regression: 

var = pf_25_vwr.columns.drop("Date")
params = pd.DataFrame(columns =["a", "b","s","h","R2/s(e)"])
tvals =  pd.DataFrame(columns =["a", "b","s","h", "R2/s(e)"])
X = np.array(OFF).reshape(len(OFF),len(OFF.columns))
X = sm.add_constant(X)
for v in var:
    y = np.array(pf_25_vwr[v]) / 100 - np.array(RF)
    linmod = sm.OLS(y,X)
    results = linmod.fit()
    coef = pd.DataFrame([np.append(results.params.round(2),results.rsquared.round(2))],
                        columns=["a", "b","s","h","R2/s(e)"])
    ttest = pd.DataFrame([np.append(results.tvalues.round(2),np.std(results.resid).round(4))],
                         columns=["a", "b","s","h", "R2/s(e)"])
    params = pd.concat([params,coef], ignore_index=True)
    tvals =  pd.concat([tvals,ttest], ignore_index=True)

#params # Relatively close - > So assume it is correct. 



C:\Users\andre\AppData\Local\Temp\ipykernel_19148\409608238.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  params = pd.concat([params,coef], ignore_index=True)
C:\Users\andre\AppData\Local\Temp\ipykernel_19148\409608238.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tvals =  pd.concat([tvals,ttest], ignore_index=True)


In [649]:
# Generate table:
def out_array(param):
    p = param
    N = np.sqrt(len(params[p]))
    parameter = np.array(params[p]).reshape((int(N),int(N)))
    ttest = np.array(tvals[p]).reshape((int(N),int(N)))
    tab = np.append(parameter,ttest,axis=1)
    tabdf = pd.DataFrame(tab, columns=pd.MultiIndex.from_tuples(
                          [(r"\beta/R^2","L"), ("","2"), ("","3"), ("","4"), ("","H"),
                           (r"t(\beta)/s(e)","L"), ("","2"), ("","3"), ("","4"), ("","H")]))
    tabdf = tabdf.set_index([pd.Index([param,"", "", "", ""]),pd.Index(["S", "2", "3", "4", "B"])])

    return tabdf

a = out_array("a")
b = out_array("b")
s = out_array("s")
h = out_array("h")
R2_se = out_array("R2/s(e)")
tab = pd.concat([a,b,s,h,R2_se])


In [650]:
latex_table = tab.to_latex(index=True, float_format="%.2f", caption="Original FF", label="tab:sample_table")
print(latex_table)

\begin{table}
\caption{Original FF}
\label{tab:sample_table}
\begin{tabular}{llrrrrrrrrrr}
\toprule
 &  & \beta/R^2 & \multicolumn{4}{r}{} & t(\beta)/s(e) & \multicolumn{4}{r}{} \\
 &  & L & 2 & 3 & 4 & H & L & 2 & 3 & 4 & H \\
\midrule
a & S & -0.00 & -0.00 & -0.00 & 0.00 & 0.00 & -3.46 & -1.10 & -0.88 & 1.02 & 0.87 \\
\cline{1-12}
\multirow[t]{4}{*}{} & 2 & -0.00 & -0.00 & 0.00 & 0.00 & 0.00 & -1.66 & -0.50 & 2.05 & 2.37 & 1.12 \\
 & 3 & -0.00 & 0.00 & 0.00 & 0.00 & 0.00 & -0.44 & 0.78 & 0.18 & 1.86 & 0.71 \\
 & 4 & 0.00 & -0.00 & -0.00 & 0.00 & 0.00 & 1.52 & -1.69 & -0.07 & 1.19 & 0.50 \\
 & B & 0.00 & -0.00 & -0.00 & -0.00 & -0.00 & 2.98 & -0.34 & -0.49 & -0.80 & -1.52 \\
\cline{1-12}
b & S & 1.04 & 0.97 & 0.94 & 0.90 & 0.95 & 39.29 & 49.91 & 58.25 & 59.85 & 58.02 \\
\cline{1-12}
\multirow[t]{4}{*}{} & 2 & 1.10 & 1.02 & 0.97 & 0.97 & 1.07 & 53.85 & 60.04 & 59.48 & 63.28 & 63.37 \\
 & 3 & 1.10 & 1.02 & 0.97 & 0.98 & 1.06 & 59.46 & 57.17 & 54.26 & 58.77 & 50.69 \\
 & 4 & 1.06 & 1.07 